In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import numpy as np
import datetime
import re
import os

In [ ]:
"""
category_list -> article_url -> pixnet_crawler 每天收集資料(熱門貼文)

假如known author uuid -> article_url_by_author -> pixnet_crawler 收集特定喜歡的作者貼文
"""

In [ ]:
user_agent = [
"Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
"Mozilla/5.0 (Windows NT 10.0; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0",
"Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; InfoPath.3; rv:11.0) like Gecko",
"Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)",
"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
"Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
"Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
"Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11",
"Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Avant Browser)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)"
]

header={'User-Agent':random.choice(user_agent)}

In [ ]:
#取得 痞客幫 category list
def get_pixnet_list():
    a=requests.get('https://www.pixnet.net/blog?utm_source=PIXNET&utm_medium=navbar&utm_term=blog',
                  headers=header,timeout=20)
    a.encoding='utf8'

    soup=BeautifulSoup(a.text,'lxml')

    C=[[s.find('a').get('href'),s.find('a').text.strip()] for s in soup.find('ul',{'id':'navigation'}).find_all('li')]
    C=[s for s in C if s[0]!= '/blog']
    
    C=pd.DataFrame(C,columns=['url','name'])
    
    C['url']='https://www.pixnet.net'+C['url']
    
    return C

C=get_pixnet_list()

In [ ]:
#痞客幫內文、留言
def pixnet_crawler(url,comment_bool=True):
    a=requests.get(url,headers=header,timeout=20)
    a.encoding='utf8'

    soup=BeautifulSoup(a.text,'lxml')

    #收集內文
    
    #id
    try:
        #多數人為這個形式
        id_part1=re.search(string=url,pattern='/([A-z0-9]+)\.pixnet\.net').group(1)
    except:
        id_part1=re.search(string=url,pattern='/([\.A-z0-9-]+)/blog').group(1)

    id_part2=re.search(string=url,pattern='post/(\d+)').group(1)

    article_id=id_part1+'-'+id_part2

    article_url=f'https://{id_part1}.pixnet.net/blog/post/{id_part2}'

    print(article_url)

    #作者唯一id、不知名類別id、分享數、留言數、留言總頁數
    info_block=soup.find('body',class_=re.compile(f'article_{id_part2}'))

    author_pixnet_id=info_block.get('data-author-member-uniqid')
    author_id=id_part1
    
    share_count=info_block.get('data-shareds-count')

    comment_count=info_block.get('data-comments-count')
    
    comment_closed_page=int(np.ceil((int(comment_count)/100)))
    #發文時間
    created_at=soup.find('ul',class_='article-head').find(class_='publish').get('content')
    created_at=created_at[:created_at.find('.')]
    created_at=datetime.datetime.strptime(created_at,'%Y-%m-%dT%H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')

    #標題
    title=soup.find('ul',class_='article-head').find('h2').text.strip()

    #推文、收藏數
    try:
        collect_count=requests.get(f'https://{id_part1}.pixnet.net/api/blog/collectionstatus?object_id=https%3A%2F%2F{id_part1}.pixnet.net%2Fblog%2Fpost%2F{id_part2}&type=article',
                     headers=header,timeout=20).json()['data']['collect_count']
    except:
        collect_count=np.nan
    
    try:
        recommend_count=requests.get(f'https://{id_part1}.pixnet.net/api/blog/articlepushcounts?article_id={id_part2}',
                     headers=header,timeout=20).json()['data']['total_recommend_count']
    except:
        recommend_count=np.nan
        
    #FB按讚數
    temp=requests.get(f'https://www.facebook.com/plugins/like.php?href=http%3A%2F%2F{id_part1}.pixnet.net%2Fblog%2Fpost%2F{id_part2}&layout=button_count&show_faces=false&width=90&action=like&font=arial&colorscheme=light&height=21',
                     headers={'User-Agent':random.choice(user_agent),
                              'Cookie':'c_user=100002615484088; xs=15%3AEAgYmtuLadf4nQ%3A2%3A1632559371%3A-1%3A11327%3A%3AAcX6sKEQKPcuYVnojBTDpPNfZCbPby0erjk_biX8FQ; fr=0pSi1ugyZpgDmAb4E.AWXXRQ3rBxIZj3817qRuMOcEtgs.BhXZkM.L_.AAA.0.0.BhXZkM.AWWLSL6sXYA; spin=r.1004530306_b.trunk_t.1633959989_s.1_v.2_',
                             },timeout=20)

    like_count=BeautifulSoup(temp.text,'lxml').find(class_='_5n6h _2pih').text
    
#     like_count=np.nan

    #作者、標籤、內文、table數、image數、影片數(youtube)
    all_table=soup.find('div',class_='article-body').find_all('table')
    table_count=len(all_table)

    img=[s for s in soup.find('div',class_='article-body').find_all(re.compile('img|picture')) if s.get('title') is not None and s.get('title') != '']
    img=[s.get('src') for s in img]
    
    all_img=[]
    
    for s in img:
        if bool(re.search(string=s,pattern='^https')):
            all_img.append(s)
        else:
            all_img.append('https:'+s)
    
    img_count=len(all_img)

    all_frame=soup.find('div',class_='article-body').find_all('iframe')
    video_count=len([s for s in all_frame if bool(re.search(string=str(s),
                                                            pattern='autoplay|youtube\.com'))])

    author=soup.find(class_='author-profile__name').text.strip()

    tag=';'.join([s.text.strip() for s in soup.find('div',class_='article-body').find_all(class_='tag')])

    #完整版內文(+表格、圖片、影片描述文字)
    temp=soup.find('div',class_='article-body')
    temp=str(temp)
    temp=temp[:temp.find('<div class="tag-container-parent">')]

    temp=BeautifulSoup(temp,'lxml')
    content_origin=temp.text.strip()

    #去除版內文
    for s in temp.find_all(re.compile('iframe|img|table')):
        s.decompose()

    content=temp.text.strip()
    
    #文章分類、個人分類
    temp=soup.find('ul',class_='refer').find_all('li')

    category=[s.text.split('：')[1] for s in temp if bool(re.search(string=str(s),pattern='全站分類'))][0]

    board=[s.find('a').get('href') for s in temp if bool(re.search(string=str(s),pattern='全站分類'))][0]
    board=re.search(string=board,pattern='articles/(.*)').group(1)
    
    self_category=[s.text.split('：')[1] for s in temp if bool(re.search(string=str(s),pattern='個人分類'))]
    
    if len(self_category) >0:
        self_category=self_category[0]
    else:
        self_category=''
    
    post=pd.DataFrame({
            'board_id':board,
            'article_id':article_id,
            'author_pixnet_id':author_pixnet_id,
            'author_id':author_id,
            'author':author,
            'created_at':created_at,
            'title':title,
            'content':content,
            'content_origin':content_origin,
            'category':category,
            'self_category':self_category,
            'collect_count':collect_count,
            'recommend_count':recommend_count,
            'comment_count':comment_count,
            'share_count':share_count,
            'like_count':like_count,
            'table_count':table_count,
            'image_count':img_count,
            'video_count':video_count,
            'tag':tag,
            'image_url':';'.join(all_img),
            'article_url':article_url,
    },index=[0])
    
    #收集留言
    C=[]
    
    cn=0
    
    for p in range(1,comment_closed_page+1):
        comment_url=f'https://{id_part1}.pixnet.net/blog/post/{id_part2}/comments?comment_page={p}'

        comment=requests.get(comment_url,
                             headers=header,timeout=20).json()

        temp=BeautifulSoup(comment['list'].strip(),'lxml')

        for c in temp.find_all(class_='single-post'):
            if 'secret' in c.get('class'):
                #自創悄悄話的unique_id
                comment_id=f'{article_id}-secret-{cn}'
                cn=cn+1
                
                author=''
                author_id=''
                author_img=''
                floor=''
                created_at=None
                content=c.find(class_='post-text').text.strip()
                reply_author=''
                reply_content=''
                reply_author_id=''
                reply_time=None
            else:
                comment_id=c.find(class_='post-info').find('a').get('name')
                author=c.find(class_='post-info').find(class_='user-name').text.strip()

                #訪客或是帳號為有連結(已被刪除?)
                try:
                    author_id=c.find(class_='post-info').find(class_='user-name').find('a').get('href')
                    author_id=re.search(string=author_id,pattern='/pcard/([A-z0-9]+)$').group(1)
                except:
                    #unknown
                    author_id=''

                floor=c.find(class_='post-info').find(class_='floor').text.strip('#')

                created_at=c.find(class_='post-info').find(class_='post-time').text.strip()
                created_at=re.search(string=created_at,pattern='\d+/\d+/\d+ \d+:\d+').group(0)
                created_at=datetime.datetime.strptime(created_at,'%Y/%m/%d %H:%M').strftime('%Y-%m-%d %H:%M:%S')

                author_img=c.find(class_='post-photo').find('img').get('src')

                content=c.find(class_='post-text').text.strip()

                if c.find(class_='reply-text') is None:
                    reply_author=''
                    reply_content=''
                    reply_author_id=''
                    reply_time=None
                else:
                    reply_author=c.find(class_='reply-text').find('a').text.strip()

                    u=c.find(class_='reply-text').find('p').find('a').get('href')
                    try:
                        #多數人為這個形式
                        reply_author_id=re.search(string=u,pattern='/([A-z0-9]+)\.pixnet\.net').group(1)
                    except:
                        reply_author_id=re.search(string=u,pattern='/([\.A-z0-9-]+)/blog').group(1)

                    t=c.find(class_='reply-text').text.strip()

                    reply_content=t.split(f'{reply_author} 於')[0].strip()

                    reply_time=re.search(string=t,pattern='\d+/\d+/\d+ \d+:\d+').group(0)
                    reply_time=datetime.datetime.strptime(reply_time,'%Y/%m/%d %H:%M').strftime('%Y-%m-%d %H:%M:%S')

            C.append(pd.DataFrame({
                'board_id':board,
                'article_id':article_id,
                'comment_id':comment_id,
                'author_id':author_id,
                'author':author,
                'author_img':author_img,
                'floor':floor,
                'created_at':created_at,
                'content':content,
                'reply_author_id':reply_author_id,
                'reply_author':reply_author,
                'reply_time':reply_time,
                'reply_content':reply_content,
                'article_url':article_url
            },index=[0]))    
    
    if len(C)>0:
        C=pd.concat(C).reset_index(drop=True)
    else:
        C=pd.DataFrame()
    
    if comment_bool:
        return post,C
    else:
        return post

In [ ]:
#抓取一般首頁文章網址(用來找熱門作者)
def get_article_url(url,page):
    au=[]
    
    for p in range(1,page+1):
        a=requests.get(f'{url}/latest/{p}',headers=header,timeout=20)
        a.encoding='utf8'
        
        soup=BeautifulSoup(a.text,'lxml')
        
        target=[s.get('href') for s in soup.find(class_='box-body').find_all('a') if s.get('href').find('blog/post') != -1]

        au.extend(target)

    return au

In [ ]:
all_u=[]

page=5

for u in C['url']:
    all_u.extend(get_article_url(url=u,page=page))

In [ ]:
def clean_url(url):
    #id
    try:
        #多數人為這個形式
        id_part1=re.search(string=url,pattern='/([A-z0-9]+)\.pixnet\.net').group(1)
    except:
        id_part1=re.search(string=url,pattern='/([\.A-z0-9-]+)/blog').group(1)

    id_part2=re.search(string=url,pattern='post/(\d+)').group(1)

    article_id=id_part1+'-'+id_part2

    article_url=f'https://{id_part1}.pixnet.net/blog/post/{id_part2}'
    
    return article_url

In [ ]:
all_u=[clean_url(s) for s in all_u]

all_u=np.unique(all_u).tolist()

In [ ]:
R=[]
Co=[]

In [ ]:
for u in all_u[0:]:
    try:
        p,c=pixnet_crawler(url=u,comment_bool=True)

        R.append(p)
        Co.append(c)
    except Exception as e:
        print(f'Error for : {u}')
        print(e,'\n\n')

In [ ]:
df=pd.concat(R)
df=df[~df.duplicated(subset=['article_id'])]
df=df.sort_values(['created_at'])

In [ ]:
Co=pd.concat(Co)
Co=Co[~Co.duplicated(subset=['article_id','comment_id'])]

In [ ]:
#download all image
def download_media(path,df):
    df=df[~df.duplicated(subset=['article_id'])]
    df=df.where(~df.isnull(),None)
    df=df.reset_index(drop=True)

    for i in range(0,df.shape[0]):
        uid=df.loc[i,'article_id']
        author=df.loc[i,'author']+'-'+df.loc[i,'author_pixnet_id']
        
        #edge_sidecar_to_children
        if df.loc[i,'image_url'] is None or df.loc[i,'image_url'] == '':
            target=[]
        else:
            target=df.loc[i,'image_url']
            target=target.split(';')

        image_target=target
        
        if len(image_target)>0:
            try:
                if not os.path.isdir(f'{path}{author}/'):
                    os.makedirs(f'{path}{author}/')
            except:
                #名子不符合磁碟要求命名規則
                author=df.loc[i,'author_pixnet_id']
                if not os.path.isdir(f'{path}{author}/'):
                    os.makedirs(f'{path}{author}/')                
        
        for i,j in enumerate(image_target):
            try:
                a=requests.get(j,headers=header,timeout=20)

                with open(path+f'{author}/{uid}_{i}.jpg','wb') as f:
                    f.write(a.content)
            except Exception as e:
                print(f'Error for url : {j}')
                print(e,'\n\n')

#download media
if len(df)>0:
    path='/self_project/self_django/self_keep/pixnet/'

    download_media(path=path,df=df)

In [ ]:
#由最新文章開始往回取幾頁，如果為None，代表爬所有文章網址
def get_article_url_by_author(author_pixnet_id,page):
    u1=f'https://www.pixnet.net/pcard/api/publish/articles?owner_uniq_id={author_pixnet_id}&per_page=10&type=0'
    
    n=1
    
    if page is None:
        page=9999
    
    deter=0
    
    U=[]
    
    while deter == 0 and n<=page:
        if n == 1:
            a=requests.get(u1,headers=header,timeout=20).json()
        else:
            u=f'https://www.pixnet.net/pcard/api/publish/articles?owner_uniq_id={author_pixnet_id}&per_page=10&type=0&page_info={page_info}'

            a=requests.get(u1,headers=header,timeout=20).json()
        
        if 'page_info' in a['data'].keys():
            page_info=a['data']['page_info']
        else:
            deter=1
        
        U.append(pd.DataFrame(a['data']['articles']).drop(['images'],axis=1))
        
        n=n+1
    
    if len(U)>0:
        return pd.concat(U).reset_index(drop=True)
    else:
        return pd.DataFrame()

In [ ]:
#加上爬蟲時間
df['create_time']=format(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S')

C['create_time']=format(datetime.datetime.now()-datetime.timedelta(hours=8),'%Y-%m-%d %H:%M:%S')

In [ ]:
from sqlalchemy import create_engine
import pymysql
import MySQLdb

path='/additional_file/sql_config.txt'

connection=pd.read_csv(path,index_col='name')

def get_sql(host,port,user,password,db_name):
    """
    host:server ip of sql
    port:port sql
    user:user name of sql
    password:password of sql user
    db_name:name of database
    """
    conn=pymysql.connect(host=host,
                         port=int(port),
                         user=user,
                         password=password,
                         db=db_name)
    cursor=conn.cursor()
    engine = create_engine('mysql+mysqldb://%s:%s@%s:%s/%s?charset=utf8mb4' % 
                           (user,password,host,port,db_name))
    return (conn,cursor,engine)

conn,cursor,engine=get_sql(connection.loc['ip','value'],int(connection.loc['port','value']),
                           connection.loc['user','value'],
                           connection.loc['password','value'],'crawler')       
